In [13]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

In [77]:
# Setup; data loading and cleaning
data_file = "/home/davidb/DEST2_contrib/output/moments_output_jackknife.tsv"
max_num_params = 8
est_cols = ['est' + str(i) for i in range(max_num_params)]

# Read in data
df = pd.read_csv(data_file, sep='\t', header=None,
                     names=['model', 'pop_of_interest'] + \
                           ['init' + str(i) for i in range(max_num_params)] + \
                           est_cols + \
                           ['upper_bound' + str(i) for i in range(max_num_params)] + \
                           ['log_likelihood', 'collapsed_pop_ll', 'func_calls', 'grad_calls',
                            'maxiter', 'hour_limit',
                            'jackknife_id', 'region'])

# Group by model, as applied to each jackknife replicate of each region
df.groupby(['region', 'model', 'pop_of_interest', 'jackknife_id'],
           dropna=False).mean()

# Get the best fit for each jackknife replicate
df = df.loc[df.groupby(['region', 'model', 'pop_of_interest', 'jackknife_id'],
                       dropna=False)\
            ['log_likelihood'].idxmax()]
df = df[['region', 'model', 'pop_of_interest', 'jackknife_id', 'log_likelihood', 'collapsed_pop_ll'] + \
        est_cols]

def calculate_ci(x):
    return (list(x.nsmallest(2))[0], 
            list(x.nlargest(2))[0])


# df = df.reset_index()
# df.rename(columns={'level_3': 'ci_bound'}, inplace=True)
df = df.replace({'pop_of_interest': {0.0: '0', 
                                     1.0 : '1',
                                     2.0 : '2',
                                     3.0 : '3',
                                     np.nan: 'NA'},
                 'ci_bound': {0: 'lower', 1: 'upper'}})

# Merge model and pop of interest, which do not need to be distinguished now that
# we're only considering models within each region, and not calling general model
# functions.
pop_of_interest_suffices = ['_' + poi if poi != 'NA' else '' for poi in df['pop_of_interest']]
df['model'] = df['model'] + pop_of_interest_suffices
df.drop(columns=['pop_of_interest'], inplace=True)

In [54]:
# Model has a significant effect on collapsed-population log-likelihood
Europe_models = df[df.region == "Europe"].model.unique()
scipy.stats.kruskal(*[df[(df.region == "Europe") & (df.model == model)].collapsed_pop_ll
                      for model in Europe_models]).pvalue

8.738345441526625e-28

In [55]:
# The split model has the greatest average collapsed-population log-likelihood for
# European models.
[(model, df[(df.region == "Europe") & (df.model == model)].collapsed_pop_ll.mean())
 for model in Europe_models]

[('admixture_0', -4.420659018597776),
 ('admixture_1', -4.4199019251863),
 ('admixture_2', -4.422253117397938),
 ('split', -4.406278823502346),
 ('twosplits_0', -4.4201607037639175),
 ('twosplits_1', -4.420270875329244),
 ('twosplits_2', -4.419368979506466)]

In [57]:
# The split model gives significantly greater collapsed-population log-likelihood
# than all each other models, indicating that the suture zone does not exist.
for model in Europe_models:
    if model == "split":
        continue

    p = scipy.stats.wilcoxon(df[(df.region == "Europe") & (df.model == "split")].collapsed_pop_ll,
                             df[(df.region == "Europe") & (df.model == model)].collapsed_pop_ll,
                             alternative='greater').pvalue
    print(model, p)

admixture_0 9.094947017729282e-13
admixture_1 9.094947017729282e-13
admixture_2 9.094947017729282e-13
twosplits_0 9.094947017729282e-13
twosplits_1 9.094947017729282e-13
twosplits_2 9.094947017729282e-13


In [64]:
# The two_epoch model gives significantly greater collapsed-population log-likelihood
# than the split model in the mainland samples, indicating that non-Caribbean American
# samples all fall into one cluster.
scipy.stats.wilcoxon(df[(df.region == "mainland") & (df.model == "two_epoch")].collapsed_pop_ll,
                        df[(df.region == "mainland") & (df.model == "split")].collapsed_pop_ll,
                        alternative='greater').pvalue

7.025937520666048e-07

In [63]:
# The two_epoch model gives significantly greater collapsed-population log-likelihood
# than the split model in the Americas, indicating that the Caribbean cluster is 
# not valid.
scipy.stats.wilcoxon(df[(df.region == "Americas") & (df.model == "two_epoch")].collapsed_pop_ll,
                        df[(df.region == "Americas") & (df.model == "split")].collapsed_pop_ll,
                        alternative='greater').pvalue

6.897607818245888e-09

In [65]:
# Model has a significant effect on collapsed-population log-likelihood for Transatlantic
# models.
Transatlantic_models = df[df.region == "Transatlantic"].model.unique()
scipy.stats.kruskal(*[df[(df.region == "Transatlantic") & (df.model == model)].collapsed_pop_ll
                      for model in Transatlantic_models]).pvalue

2.787106945482969e-31

In [90]:
# The admixture_2 model, i.e. Americas as an admixture of Eastern Europe and Zambia,
# has the greatest average collapsed-population log-likelihood for the Transatlantic
# region.
[(model, df[(df.region == "Transatlantic") & (df.model == model)].log_likelihood.mean())
 for model in Transatlantic_models]

[('admixture_0', -6.390618788793853),
 ('admixture_1', -6.2558477369473255),
 ('admixture_2', -6.059614888268547),
 ('admixture_3', -6.338153631689331)]

In [91]:
# The admixture_2 model gives significantly greater collapsed-population log-likelihood
# than all each other model, indicating that American flies are better described
# as an admixture of Zambia and Eastern Europe than as any other combination of
# Western or Eastern Europe and Guinea or Zambia.
putative_best_model = "admixture_2"
for model in Transatlantic_models:
    if model == putative_best_model:
        continue

    p = scipy.stats.wilcoxon(df[(df.region == "Transatlantic") & (df.model == putative_best_model)].collapsed_pop_ll,
                             df[(df.region == "Transatlantic") & (df.model == model)].collapsed_pop_ll,
                             alternative='greater').pvalue
    print(model, p)

admixture_0 9.094947017729282e-13
admixture_1 9.094947017729282e-13
admixture_3 9.094947017729282e-13


In [105]:
print("95% CIs for parameter estimates for split model of Europe:")
for i, param_name in enumerate(['nu_EUW', 'nu_EUE', 'T_split', 'm']):
    ci = np.array(df[(df.region == "Europe") & (df.model == "split")]['est' + str(i)].quantile([0.025, 0.975]))
    print(f"{param_name}\t{ci}")

95% CIs for parameter estimates for split model of Europe:
nu_EUW	[2.82002992 2.94922152]
nu_EUE	[0.58929685 0.64438409]
T_split	[0.03808484 0.04016409]
m	[16.6489238  18.02249604]


In [109]:
print("95% CIs for parameter estimates for two_epoch model of mainland Americas:")
for i, param_name in enumerate(['N', 'T']):
    ci = np.array(df[(df.region == "mainland") & (df.model == "two_epoch")]['est' + str(i)].quantile([0.025, 0.975]))
    print(f"{param_name}\t{ci}")

95% CIs for parameter estimates for two_epoch model of mainland Americas:
N	[1.85794596 2.10597247]
T	[0.02297162 0.02874046]


In [110]:
print("95% CIs for parameter estimates for two_epoch model of the Americas:")
for i, param_name in enumerate(['N', 'T']):
    ci = np.array(df[(df.region == "Americas") & (df.model == "two_epoch")]['est' + str(i)].quantile([0.025, 0.975]))
    print(f"{param_name}\t{ci}")

95% CIs for parameter estimates for two_epoch model of the Americas:
N	[1.58762755 1.8860231 ]
T	[0.01854634 0.03073693]


In [112]:
# p_admix is the proportion of genetic material from the African population in the
# admixture event.
print("95% CIs for parameter estimates for admixture model of the Americas as admixture of EUE and Zambia:")
for i, param_name in enumerate(['nu_Afr', 'nu_EUE', 'nu_Am', 'T_split', 'T_admix', 'm2', 'm3', 'p_admix']):
    ci = np.array(df[(df.region == "Transatlantic") & (df.model == "admixture_2")]['est' + str(i)].quantile([0.025, 0.975]))
    print(f"{param_name}\t{ci}")

95% CIs for parameter estimates for admixture model of the Americas as admixture of EUE and Zambia:
nu_Afr	[0.36379114 0.39992383]
nu_EUE	[0.69469272 0.74367556]
nu_Am	[0.79505027 0.96947208]
T_split	[0.15644274 0.83881196]
T_admix	[0.01856334 0.02118287]
m2	[1.00367891e-03 2.15084793e+00]
m3	[12.32206064 16.13708399]
p_adimx	[0.79433422 0.82854144]
